<DIV ALIGN=CENTER>

# Introduction to Cassandra
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

In the previous course, we discussed relational databases, SQL, and
using Python to work with relational databases. With the rapid growth
in large data sets, however, there has been an explosion in new database
technologies. In this IPython Notebook, we explore [MongoDB][mdb], one
of the more popular new database technologies.  [MongoDB][mdbw] is a
NoSQL document-oriented database, which means it is _not only SQL_ and
stores data as documents. The data are stored using dynamic schemas that
employ _BSON_ format, which is JSON-like format. For more information,
the [MongoDB documentation website][mdbd] provides a wealth of useful
information.

-----
[mdb]: https://www.mongodb.org
[mdbw]: https://en.wikipedia.org/wiki/MongoDB
[mdbd]: https://docs.mongodb.org/manual/

## Python with MongoDB

To use Python to interact with MongoDB, we need to use a suitable Python
library. The recommended Python library is [_pymongo_][pymdb], which
provides support for establishing a connection between a Python program
and a MongoDB server as well as support tools for working with MongoDB. 

We have already installed _pymongo_ in the course Docker container;
however, you can easily install is by using `pip`, for example to
install _pymongo_ for use with Python3 for the current user, we can
execute:

```console
pip3 install pymongo --user
```

Once this library is installed, we can import the MongoDB client to
establish a connection and retrieve data and MongoDB information.

```python
from pymongo import MongoClient
```

-----

[pymdb]: http://api.mongodb.org/python/current/

!pip install cassandra-driver --user

In [1]:
from cassandra.cluster import Cluster

-----

## Remote Cassandra Server

To connect to a remote Cassandra server, for instance by using the course
cluster system, we simply need the IP address for the server and the
port number on which the Cassandra daemon is listening. For this course,
Notebooks running on the course JupyterHub Server can access a Cassandra
server, hosted by Micrsoft Azure, on `40.124.12.119` on the default port of `9042`:


```python
cluster = Cluster(['40.124.12.119'])
```

-----

In [2]:
cassandra_ips = ['40.124.12.119']

# Establish a connection to Cassandra

# The Policy is necessary to allow Cassandra to run on Azure.
from cassandra.policies import WhiteListRoundRobinPolicy
pcy = WhiteListRoundRobinPolicy(cassandra_ips)

# Create Connection
cluster = Cluster(contact_points=cassandra_ips, load_balancing_policy=pcy)
session = cluster.connect()

print('Cluster Name: {0}'.format(cluster.metadata.cluster_name))
for host in cluster.metadata.all_hosts():
    print('{0}: Host: {1} in {2}'\
          .format(host.datacenter, host.address, host.rack))

Cluster Name: lcdm
datacenter1: Host: 10.3.0.5 in rack1
datacenter1: Host: 40.124.12.119 in rack1


-----

## Cassandra Keyspace

-----

In [3]:
# Filename containing user's netid
fname = '/home/data_scientist/users.txt'
with open(fname, 'r') as fin:
    netid = fin.readline().rstrip()

# We will delete our working directory if it exists before recreating.
ks_name = '{0}'.format(netid)

### Creating Our Keyspace

Each student will have their own keyspace. TO do this, the following code cell uses Python code to first drop the user's keyspace if it exists, and to then create the user's keyspace. This effectively deletes all data added to the keyspace from this Notebook. Alternatively, we could employ CQL (Cassandra Query Language) to perform the same steps:

```python
# CQL to drop named keyspace
drop_ks = '''
DROP KEYSPACE IF EXISTS {0} ;
'''

# CQL to create named keyspace (We need to escape curly braces)
create_ks = '''
CREATE KEYSPACE {0} \
    WITH REPLICATION = {{'class': 'SimpleStrategy', 'replication_factor': 1}} ;
'''

# Execute CQL for Keyspaces
session.execute(drop_ks.format(ks_name))
session.execute(create_ks.format(ks_name))

# Display Keyspace names
keys = [val for val in sorted(cluster.metadata.keyspaces.keys()) if 'system' not in val]
for ks in keys:
    print(ks)
```

In [4]:
# Boilerplate code we need to create keyspace from within Python
# Altenaitve is to create keyspace from CQL.

from cassandra.query import dict_factory
session.row_factory = dict_factory

from cassandra.cqlengine import connection, management
connection.set_session(session)

# Explicitly set session hosts, this removes annoying warnings.
connection.session.hosts = cassandra_ips

# Drop Keyspace if it exists
if ks_name in cluster.metadata.keyspaces:
    management.drop_keyspace(ks_name)

# Create Keyspace
management.create_keyspace_simple(ks_name, 1)

# Set keyspace for this session
# Note: If keyspace exists in Cassandra instance, this is only line we need.
session.set_keyspace(ks_name)

# Display all non-system keyspaces.
# Do not change to a different keyspace!

keys = [val for val in sorted(cluster.metadata.keyspaces.keys()) if 'system' not in val]
for ks in keys:
    print(ks)

print('CQL Query to recreate this keyspace:')
print(40*'-')
print(cluster.metadata.keyspaces[ks_name].export_as_string())

/home/data_scientist/.local/lib/python3.5/site-packages/cassandra/cqlengine/management.py:431: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


bigdog
info490
CQL Query to recreate this keyspace:
----------------------------------------
CREATE KEYSPACE bigdog WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}  AND durable_writes = true;


----

### Schema Creation


-----

In [5]:
drop_schema = '''
DROP TABLE IF EXISTS students ;
'''

create_schema = '''
CREATE TABLE students (
    id int,
    fname text,
    lname text,
    company text,
    hire_date timestamp,
    PRIMARY KEY(lname, id)
);
'''
print('\nDeleting Table')
print(40*'-')
d_result = session.execute(drop_schema)
print(d_result.response_future)

print('\nCreating Table')
print(40*'-')
c_result = session.execute(create_schema)
print(c_result.response_future)


Deleting Table
----------------------------------------
<ResponseFuture: query='<SimpleStatement query="
DROP TABLE IF EXISTS students ;
", consistency=Not Set>' request_id=18 result=None exception=None host=40.124.12.119>

Creating Table
----------------------------------------
<ResponseFuture: query='<SimpleStatement query="
CREATE TABLE students (
    id int,
    fname text,
    lname text,
    company text,
    hire_date timestamp,
    PRIMARY KEY(lname, id)
);
", consistency=Not Set>' request_id=19 result=None exception=None host=40.124.12.119>


-----

## Adding Data

Given a collection, we can easily add new _documents_ to our MongoDB
collection by employing a Python dictionary to map the document schema
to the document data. In the following code cell, we first create a
`student` document, followed by a `students` collection to hold
`student` documents, and we insert the first student by using the
`insert_one` method on the `students` collection. We retrieve this new
students id, which we display as a validation of this process. After
this process, we display the newly created database and collection.

-----

In [6]:
import uuid
from datetime import datetime

student = {'id': 1,
           'fname': 'Jane',
           'lname': 'Doe',
           'company': 'bdg surf shop',
           'hire_date': datetime.now()}

insert_one = '''
INSERT INTO students (id, fname, lname, company, hire_date) 
VALUES (%(id)s, %(fname)s, %(lname)s, %(company)s, %(hire_date)s) ;
'''

io_result = session.execute(insert_one, student) 

print(io_result.response_future)

<ResponseFuture: query='<SimpleStatement query="
INSERT INTO students (id, fname, lname, company, hire_date) 
VALUES (%(id)s, %(fname)s, %(lname)s, %(company)s, %(hire_date)s) ;
", consistency=Not Set>' request_id=29 result=None exception=None host=40.124.12.119>


In [7]:
insert_many = '''
INSERT INTO students (id, fname, lname, company, hire_date) 
VALUES (:id, :fname, :lname, :company, :hire_date) ;
'''

im_prepared = session.prepare(insert_many)

-----

Unlike relational database tables, a MongoDB collection can store
documents that have different schema. We demonstrate this in the next
two code cells where we create two new students that each have different
schema from the original student. Atfer inserting these new students, we
count the number of documents in the `students` collection.

-----

In [8]:
new_students = [
    {'fname': 'Mike',
     'lname': 'Simone',
     'company': 'Del Ray Enterprises'},
    {'fname': 'Clair',
     'lname': 'Hwu',
     'company': 'Hoboken Surfware Incorporated'}]

# Insert multiple entires
for idx, student in enumerate(new_students):
    im_result = session.execute(im_prepared, ((idx + 1), 
                                              student['fname'], student['lname'],
                                              student['company'], datetime.now()))
    
# Insert one using direct value substitution
im_result = session.execute(im_prepared, (3, 'John', 'Doe', "Bigdog's Surf Shop",
                                          datetime.now()))

-----

### Retrieving Data



-----

In [9]:
qr = session.execute('SELECT COUNT(*) FROM students')

print(qr.response_future)

print(40*'-')
print('Employee count = {0}'.format(qr[0]['count']))

<ResponseFuture: query='<SimpleStatement query="SELECT COUNT(*) FROM students", consistency=Not Set>' request_id=34 result=[{'count': 4}] exception=None host=40.124.12.119>
----------------------------------------
Employee count = 4


In [10]:
query = '''
SELECT fname, lname, company, hire_date FROM  students ;
'''

query_result = session.execute(query)

# Display query results
print(40*'-')
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))
    print(40*'-')

----------------------------------------
Name: Jane Doe
Company: bdg surf shop
Hired on 2016-04-12 21:58:38.453999
----------------------------------------
Name: John Doe
Company: Bigdog's Surf Shop
Hired on 2016-04-12 21:58:38.696000
----------------------------------------
Name: Clair Hwu
Company: Hoboken Surfware Incorporated
Hired on 2016-04-12 21:58:38.641000
----------------------------------------
Name: Mike Simone
Company: Del Ray Enterprises
Hired on 2016-04-12 21:58:38.585000
----------------------------------------


In [11]:
# We can easily query on Primary Keys
query = '''
SELECT fname, lname, company, hire_date 
FROM  students 
WHERE lname = %(ln)s ;
'''

query_result = session.execute(query, {'ln': 'Doe'})

# Display query results
print(40*'-')
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))
    print(40*'-')

----------------------------------------
Name: Jane Doe
Company: bdg surf shop
Hired on 2016-04-12 21:58:38.453999
----------------------------------------
Name: John Doe
Company: Bigdog's Surf Shop
Hired on 2016-04-12 21:58:38.696000
----------------------------------------


-----

### Modifing Data


-----

In [12]:
temp_students = [{'id': 11, 'fname': 'Petr', 'lname': 'Dow', 'company': 'bdg surf shop'},
                 {'id': 12, 'fname': 'Paul', 'lname': 'Dow', 'company': 'bdg surf shop'},
                 {'id': 13, 'fname': 'Mary', 'lname': 'Dow', 'company': 'bdg surf shop'},
                 {'id': 14, 'fname': 'Arthur', 'lname': 'Dow', 'company': 'bdg surf shop'}]

# Insert multiple entires
for student in temp_students:
    im_result = session.execute(im_prepared, (student['id'], 
                                              student['fname'], student['lname'],
                                              student['company'], datetime.now()))

# Count how many students
qr = session.execute('SELECT COUNT(*) FROM students')

print(40*'-')
print('Employee count = {0}'.format(qr[0]['count']))

----------------------------------------
Employee count = 8


In [13]:
ud_stmt = '''
UPDATE students
  SET fname = 'Peter'
  WHERE lname = 'Dow' AND id = 11 ;
'''

ur = session.execute(ud_stmt)

# Display Updated Row
query_result = session.execute(query, {'ln': 'Dow'})

# Display query results
for row in query_result:
    if row['fname'] == 'Peter':
        print('Name: {0} {1}'.format(row['fname'], row['lname']))
        print('Company: {0}'.format(row['company']))
        print('Hired on {0}'.format(row['hire_date']))    

Name: Peter Dow
Company: bdg surf shop
Hired on 2016-04-12 21:58:39.019999


-----



-----

In [14]:
ud_stmt = '''
UPDATE students
  SET company = 'Bigdogs Surf Shop'
  WHERE lname = 'Dow' AND id IN (11, 12, 13, 14) ;
'''

ur = session.execute(ud_stmt)

# Display Updated Rows
query_result = session.execute(query, {'ln': 'Dow'})

# Display query results
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))    
    print(40*'-')

Name: Peter Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.019999
----------------------------------------
Name: Paul Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.075999
----------------------------------------
Name: Mary Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.135999
----------------------------------------
Name: Arthur Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.191999
----------------------------------------


-----

### Deleting Data


-----

In [15]:
del_stmt = '''
DELETE FROM students
WHERE lname = 'Dow' AND id = 11 ;
'''

dr = session.execute(del_stmt)
                     
# Display Updated Rows
query_result = session.execute(query, {'ln': 'Dow'})

# Display query results
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))    
    print(40*'-')


Name: Paul Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.075999
----------------------------------------
Name: Mary Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.135999
----------------------------------------
Name: Arthur Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-12 21:58:39.191999
----------------------------------------


In [16]:
# Count how many students
qr = session.execute("SELECT COUNT(*) FROM students WHERE lname = 'Dow'")

print(40*'-')
print('Pre-Delete')
print('Employee (Dow) count = {0}'.format(qr[0]['count']))                

del_stmt = '''
DELETE FROM students
WHERE lname = 'Dow' AND id = %(id)s ;
'''

for id in [12, 13, 14]:
    dr = session.execute(del_stmt, {'id': id})
                     
# Count how many students
qr = session.execute("SELECT COUNT(*) FROM students WHERE lname = 'Dow'")

print(40*'-')
print('Post-Delete')
print('Employee (Dow) count = {0}'.format(qr[0]['count']))                

----------------------------------------
Pre-Delete
Employee (Dow) count = 3
----------------------------------------
Post-Delete
Employee (Dow) count = 0


In [17]:
print('CQL Query to recreate this keyspace:')
print(40*'-')
print(cluster.metadata.keyspaces[ks_name].export_as_string())

CQL Query to recreate this keyspace:
----------------------------------------
CREATE KEYSPACE bigdog WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}  AND durable_writes = true;

CREATE TABLE bigdog.students (
    lname text,
    id int,
    company text,
    fname text,
    hire_date timestamp,
    PRIMARY KEY (lname, id)
) WITH CLUSTERING ORDER BY (id ASC)
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND dclocal_read_repair_chance = 0.1
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interva

-----
### Student Activity

In the preceding cells, we introduced Hadoop map/reduce by using a simple word count task. Now that you have run the Notebook, go back and make the following changes to see how the results change.

1. We ignored punctuation, modify the original mapper Python code to token on white space or punctuation. How does this change the Python map-reduce output?
2. Try downloading a different text from Project Gutenberg. Can you make your map-reduce application run across multiple texts?
3. Can you make your map-reduce code compute bi-grams instead of unigrams?

-----